In [27]:
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np
from openpyxl.workbook import Workbook
import base64

In [49]:
name_ = []
publisher_ = []
setting_ = []
revenue_ = []
lifetime_revenue_ = []
downloads_ = []
lifetime_downloads_ = []
rpd_ = []
realeses_ = []
raiting_ = []

In [50]:
#функция для парсинга top grossing
def parcing_link(link):
    #опция, чтобы браузер не открывался
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    sleep(3)
    driver.get(link)
    #чтобы сайт успел прогрузится ждем 4 секунды перед парсингом
    sleep(6)
    #берем название игры
    name = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[1]/div')
    name = name.text
    name_.append(name)
    #берем паблишера
    publisher = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/div[2]')
    publisher = publisher.text
    publisher_.append(publisher)
    #пытаемся найти сетинг на странице
    try: 
        setting = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[1]/div[1]/div[2]/application-tags/div[4]/div[2]')
        setting = setting.text
    except NoSuchElementException:
        setting = '-'
    setting_.append(setting)
    #берем ревенью за 30 дней
    revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div/div')
    if revenue.text == 'No revenue over lifetime':
        revenue = 'No revenue over lifetime'
    else: 
        revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
        revenue = revenue.text  
        sleep(1)
        #находим и кликаем на суммарное ревенью
        lifetime_revenue = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/div/div[2]')
        lifetime_revenue.click()
        sleep(1)
        #далее берем информацию о lifetime revenue
        lifetime_revenue_value = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[1]/metric-widget/div/div[1]/span/span')
        lifetime_revenue_value = lifetime_revenue_value.text
    revenue_.append(revenue)
    lifetime_revenue_.append(lifetime_revenue_value)
    #берем установки за 30 дней
    downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget")
    if downloads.text == 'No downloads over lifetime':
        downloads = 'No downloads over lifetime'
        lifetime_downloads_value = 'No downloads over lifetime'
    else:
        downloads = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div[1]/span/span")
        downloads = downloads.text
        sleep(1)
        #находим и кликаем на суммарные скачивания
        lifetime_downloads = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div/div/div[2]')
        lifetime_downloads.click()
        sleep(1)
        #берем информацию о суммарных скачиваниях
        lifetime_downloads_value = driver.find_element("xpath", "/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[1]/div[2]/metric-widget/div/div/span/span")
        lifetime_downloads_value = lifetime_downloads_value.text
    downloads_.append(downloads)
    lifetime_downloads_.append(lifetime_downloads_value)
    #берем rpd
    rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget')
    if rpd.text == 'RpD — $0':
        rpd = '$0'
    else:
        rpd = driver.find_element("xpath", '/html/body/app-root/layout/div[2]/app-page/app-info-dialog/div[1]/div[3]/widgets-wrap/div/div[2]/div[2]/ltv-widget/div[2]')
        rpd = rpd.text
        rpd = rpd.replace('\n', ' ')
    rpd_.append(rpd)
    #берем дату релиза
    realeses = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/div[3]/app-description/div[1]/span[2]')
    realeses = realeses.text
    realeses_.append(realeses)
    #берем общий рейтинг
    try:
        raiting = driver.find_element("xpath", '//*[@id="mat-tab-content-0-0"]/div/info/app-rate-info/div[1]/div[1]')
        raiting = raiting.text
    except NoSuchElementException:
        raiting = '-'
    raiting_.append(raiting)

In [51]:
%%time
parcing_link('https://appmagic.rocks/iphone/genshin-impact/1467190251')

CPU times: total: 46.9 ms
Wall time: 20.6 s


In [52]:
#создаем датафрейм top free
df = pd.DataFrame({'Game_Name': name_, 'Publisher':publisher_, "Setting":setting_, \
                  "Revenue": revenue_, "Lifetime_revenue": lifetime_revenue_, "Downloads":downloads_, \
                  "Lifetime_downloads": lifetime_downloads_, "Rpd": rpd_, "Realese_date":realeses_, "Raiting": raiting_})             

In [53]:
df

,Game_Name,Publisher,Setting,Revenue,Lifetime_revenue,Downloads,Lifetime_downloads,Rpd,Realese_date,Raiting
0,Genshin Impact,miHoYo Games,Fantasy,> $100 000 000,> $2 000 000 000,> 2 000 000,> 100 000 000,"Tier-1 East $42,25 Tier-1 West $26,31 Global $...","Release Date — September 24, 2020",4.4


In [83]:
#грузим в файл excel с названием 'top_free'
df.to_excel('apps.xlsx', sheet_name='app_info', index=False)